# Patient Triaging with Quantum ML

Patient Triaging is a hybrid quantum computing-based approach for categorizing patients' triage for emergency assessment and priority treatment. The solution leverages unsupervised machine learning approach and uses patient vitals to cluster them into categories and further labels the clusters into triage priorities. This solution improves risk assesment and empowers healthcare facilities to optimize resource allocation, reduce wait times, and enhance the overall quality of care.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Patient Triaging with Quantum ML using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Patient Triaging with Quantum ML. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Patient Triaging with Quantum ML.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn=''

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
from zipfile import ZipFile
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='quantum-patient-triage-categorization'

content_type='application/zip'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [6]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

1. Supported content types: 'zip' file only with file name 'input.zip'.The zip file includes three 'csv' files with following name and information.

   a. 'Patient_Unlabelled_Data.csv' : This csv file contains patient's information, without any labels/categories. The objective of the solution is to label these patients based on their health conditions and also using information on labels/categories mentioned in 'Patient_Labelled_Data.csv'.
    
   b. 'Patient_Labelled_Data.csv' : This csv file contains patient's information with labels/categories. These labelled patients should be those who have been previously categorised by triage systems in emergency departments, and should be good representation of each label/category. Note: The quality of result depends on this labelled information.
    
   c. 'User_Input.csv': This csv file contains information on user's credentials for acessing Dwave Leap,a quantum cloud service providing real-time access to quantum computer and quantum hybrid solver service.
   

2. Mandatory Fields:

    a. 'Patient_Unlabelled_Data.csv': age, gender, chest pain type, cholesterol, max heart rate, exercise-induced angina, blood glucose, bmi, hypertension, heart_disease, diastolic blood pressure, and systolic blood pressure.
    
    b. 'Patient_Labelled_Data.csv': age, gender, chest pain type,  cholesterol, max heart rate, exercise-induced angina, blood glucose, bmi, hypertension, heart_disease, diastolic blood pressure, systolic blood pressure and triage.
    
    c. 'User_Input.csv': dwave_sapi_token, number_of_runs.
    

3. Input field descriptions:

    a. 'age' : Age of the patient (Numerical).
    
    b. 'gender' : Gender of the patient (Discrete). If the gender is 'male', then the value is '1', Else '0'.
    
    c. 'chest pain type' : Patient's severity of the chest pain, if encountered (Discrete). If no chest pain is identified then the value is '0', Else the value is between '1' and '4' based on the severity.
    
    d. 'cholesterol': Cholesterol level of patient measured in mg/dL (Numerical). 
    
    e. 'max heart rate': Maximum heart rate of patient measured in bpm.
    
    f. 'exercise-induced angina': Patient's encountering of angina due to exertion (Discrete). If encountered then the value is '1', Else '0'.
    
    g. 'blood glucose': The blood glucose level of patient in mg/dL (Numerical).
    
    h. 'bmi': The body mass index of patient (Numerical).
    
    i. 'hypertension': The flag if patient is identified with hypertension or not.If the hypertension is identified, then the value is '1', Else '0'.
    
    j. 'heart_disease': Patient's history of heart disease. If the patient has a history of heart disease the value is '1', else '0'.
    
    h. 'diastolic blood pressure': The diastolic blood pressure of patient (Numerical).
    
    k. 'systolic blood pressure': The systolic blood pressure of patient (Numerical).
    
    l. 'triage': The label/category of patient in order of the priority or treatment assesment. The values can be any labels as per user choice. For example the field value can be 'green' if least priority is to be provided or 'red' if highest priority order is to be provided.
    
    m. 'dwave_sapi_token' : The user secure API token of Dwave Leap quantum cloud service. For example, 'dwave_sapi_token'= 'DEV-************************'. This API token is provided after registering and subscribing to Dwave Leap cloud service.
    
    n. 'number_of_runs' : The number of runs a Dwave solver should be iterated through, to get the desired results (Numerical).
    

In [7]:
file_name = 'input.zip'

#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'application/zip' --region us-east-2 results.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file (in json format) contains the following files:

    1. 'results.csv': The results file contains the unlabelled csv file with predicted_label columns added as triage category of patients.


In [9]:
import json
file_path = os.getcwd()
file_name_output = 'results.csv'

results= pd.read_csv(file_name_output)

In [11]:
results.head()

,age,gender,cholesterol,max heart rate,exercise-induced angina,blood glucose,bmi,hypertension,heart_disease,diastolic blood pressure,systolic blood pressure,chest pain type_1.0,chest pain type_2.0,chest pain type_3.0,chest pain type_4.0,predicted_label
0,58.0,1.0,169.0,182.0,0.0,77.00,23.6,0.0,0.0,72.0,92.0,0.0,0.0,0.0,0.0,yellow
1,55.0,1.0,159.0,175.0,0.0,73.57,28.0,0.0,0.0,118.0,138.0,0.0,0.0,0.0,0.0,yellow
2,56.0,1.0,186.0,140.0,1.0,124.00,19.0,0.0,0.0,110.0,130.0,0.0,0.0,0.0,1.0,yellow
3,53.0,1.0,188.0,142.0,0.0,79.33,15.0,0.0,0.0,129.0,149.0,0.0,0.0,0.0,0.0,orange
4,49.0,1.0,177.0,185.0,0.0,175.74,45.4,0.0,0.0,102.0,122.0,0.0,0.0,0.0,0.0,yellow


Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
transform_input

In [15]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

......................... * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
 * Restarting with stat
 * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://169.254.255.131:8080
#033Press CTRL+C to quit#033
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 560-116-351
 * Debugger is active!
 * Debugger PIN: 560-116-351
169.254.255

In [ ]:
#output is available on following path
transformer.output_path

In [17]:
s3_conn = boto3.client("s3")
bucket_name=""
with open('output2.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [18]:
file_path = os.getcwd()
output_df_path = os.path.join(file_path, 'output2.csv')
print("Sample Output:")
output_df = pd.read_csv(output_df_path)

Sample Output:


In [20]:
output_df.head()

,age,gender,cholesterol,max heart rate,exercise-induced angina,blood glucose,bmi,hypertension,heart_disease,diastolic blood pressure,systolic blood pressure,chest pain type_1.0,chest pain type_2.0,chest pain type_3.0,chest pain type_4.0,predicted_label
0,58.0,1.0,169.0,182.0,0.0,77.00,23.6,0.0,0.0,72.0,92.0,0.0,0.0,0.0,0.0,yellow
1,55.0,1.0,159.0,175.0,0.0,73.57,28.0,0.0,0.0,118.0,138.0,0.0,0.0,0.0,0.0,yellow
2,56.0,1.0,186.0,140.0,1.0,124.00,19.0,0.0,0.0,110.0,130.0,0.0,0.0,0.0,1.0,yellow
3,53.0,1.0,188.0,142.0,0.0,79.33,15.0,0.0,0.0,129.0,149.0,0.0,0.0,0.0,0.0,yellow
4,49.0,1.0,177.0,185.0,0.0,175.74,45.4,0.0,0.0,102.0,122.0,0.0,0.0,0.0,0.0,yellow


### 4. Clean-up

#### A. Delete the model

In [21]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.